In [1]:
!pip install tensorflow==2.12.0 #2.3.1
!pip install tensorflow-gpu==2.12.0 #2.3.1
!pip install keras==2.12.0 #2.4.3
!pip install keras-vggface==0.6
!pip install keras_applications==1.0.8

ERROR: Invalid requirement: '#2.3.1'

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#2.3.1'

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#2.4.3'

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from keras_vggface.vggface import VGGFace #from keras.utils.layer_utils import get_source_inputs
from keras.models import Sequential 
from keras.layers import Activation, Dense
from keras.models import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import CSVLogger,EarlyStopping
from keras_vggface.utils import decode_predictions
import itertools

In [11]:
def load_dataset(dataset_Path,batch_size):


  train_data_dir='D:/Programs/Project_new_vgg/dataset/train'
  test_data_dir = 'D:/Programs/Project_new_vgg/dataset/test'
  val_data_dir = 'D:/Programs/Project_new_vgg/dataset/val'



  train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   rotation_range=40,
                                   
                                   zoom_range=0.2,
                                  
                                   horizontal_flip=True)

  train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        target_size=(224,224),
                        batch_size=batch_size,
                        class_mode='categorical')

  print(train_generator.classes)
  print(train_generator.class_indices)


  val_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                     rotation_range=40,   
                                   zoom_range=0.2,
                                   horizontal_flip=True)
  val_generator = val_datagen.flow_from_directory(
                        val_data_dir,
                      
                        target_size=(224,224),
                        batch_size=batch_size,
                        class_mode='categorical')


  print(val_generator.classes)
  print(val_generator.class_indices)





  test_datagen = ImageDataGenerator(rescale=1./255)

  test_generator = test_datagen.flow_from_directory(
                        test_data_dir,
                        target_size=(224,224),
                        batch_size=batch_size,
                        shuffle=False,
                        seed=42,
                        class_mode='categorical')


  print(test_generator.classes)
  print(test_generator.class_indices)
  return train_generator,val_generator,test_generator

In [12]:
def create_model():
  vggface = VGGFace(model='vgg16')
  vggface.summary()
  return vggface

In [13]:
def fine_tune(old_model):

  inp = old_model.input     # make a reference to VGG's input layer
  num_classes=75

  new_classification_layer=Dense(num_classes,activation='softmax')
  new_model=Sequential()

  # connect the above new layer to the second to last layer in VGG, and make a reference to it
  out = new_classification_layer(old_model.layers[-2].output)

# create a new network between inp and out
  model_new = Model(inp, out)

  # make all layers untrainable by freezing weights 
  for l, layer in enumerate(model_new.layers):
    layer.trainable = False
  # Enable the last three dense layers for  training
  for l, layer in enumerate(model_new.layers[-6:]):
    layer.trainable = True

  model_new.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.RMSprop(lr=1e-5),
                    metrics=['accuracy'])

  
  return model_new

In [14]:
def train_fine_tuned_vgg16_model(model_new, load_weight,train_data,val_data,batch_size,checkpoint_path):

  #%load_ext tensorboard
  #log_dir = "/content/drive/MyDrive/ICDFD_VGG/log_imfdb_dense/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  #%tensorboard --logdir '/content/drive/MyDrive/ICDFD_VGG/log_imfdb_dense/'
  #log_dir = "/content/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  #%tensorboard --logdir '/content/logs/'
  #Model loading
  if load_weight==True:
    model_new.load_weights("D:/Programs/Project_new_vgg/logs/Augmented_IIITD_new_60_20_0_V3-0002.h5")  # load the latest model and change the checkpoint name and retrain where u left off

  # Create a callback that saves the model's weights
  cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                                                 monitor='val_loss',
                                                 save_weights_only=True,
                                                 save_best_only=True,
                                                 mode='min',
                                                 period=1)
  

  csv_logger=CSVLogger('D:/Programs/Project_new_vgg/DAR_60_20_20_V2.csv',separator=',',append=True)

  steps_per_epoch =  np.floor(train_data.n / batch_size)     # total number of images in train /batch size
  validation_steps =  np.floor(val_data.n / batch_size)     # total number of images in test /batch size

  nepochs = 150   # earlier 150
   
 


  early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)  #erlier val_loss
  
  #tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

  model_new.fit_generator(train_data,
                          epochs=nepochs,      #steps_per_epoch=steps_per_epoch,
                          validation_data=val_data,
                          validation_steps=validation_steps,
                          callbacks=[cp_callback,early_stop,csv_logger])#,cp_callback,tensorboard_callback])



  model_new.save('VGGFace_mtcnn_Augmented_IIITD_Model_60_20_20_V1.h5')
  model_new.save_weights('VGGFace_mtcnn_Augmented_IIITD_finetuned_weights_60_20_20_V1.h5')
 # np.save('/content/drive/My Drive/my_history_Cov_unfrezzed_.npy',model_new.history)
  return model_new

In [15]:
model=create_model()
model_new=fine_tune(model)
model_new.summary()

Model: "vggface_vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                                 
 pool2 (MaxPooling2D)        (None, 56, 56, 128)     

In [16]:
batch_size=32
dataset_path="D:\Programs\Project_new_vgg\dataset"
root_dir="D:\Programs\Project_new_vgg"

train_data,val_data,test_data=load_dataset(dataset_path,batch_size)
for names in model_new.layers:
  if names.trainable==True:
    print(names.name)
load_epoch_weights= False #True  # if True change the below checkpoint name before training.
checkpoint_path = root_dir+"/logs/Augmented_IIITD_new_60_20_0_V3-{epoch:04d}.h5"     #earlier
#checkpoint_path = dataset_path+"/logs/ICDFD_mtcnn_VGGFace_new-{epoch:04d}.h5"

Found 9382 images belonging to 75 classes.
[ 0  0  0 ... 74 74 74]
{'P1': 0, 'P10': 1, 'P11': 2, 'P12': 3, 'P13': 4, 'P14': 5, 'P15': 6, 'P16': 7, 'P17': 8, 'P18': 9, 'P19': 10, 'P2': 11, 'P20': 12, 'P21': 13, 'P22': 14, 'P23': 15, 'P24': 16, 'P25': 17, 'P26': 18, 'P27': 19, 'P28': 20, 'P29': 21, 'P3': 22, 'P30': 23, 'P31': 24, 'P32': 25, 'P33': 26, 'P34': 27, 'P35': 28, 'P36': 29, 'P37': 30, 'P38': 31, 'P39': 32, 'P4': 33, 'P40': 34, 'P41': 35, 'P42': 36, 'P43': 37, 'P44': 38, 'P45': 39, 'P46': 40, 'P47': 41, 'P48': 42, 'P49': 43, 'P5': 44, 'P50': 45, 'P51': 46, 'P52': 47, 'P53': 48, 'P54': 49, 'P55': 50, 'P56': 51, 'P57': 52, 'P58': 53, 'P59': 54, 'P6': 55, 'P60': 56, 'P61': 57, 'P62': 58, 'P63': 59, 'P64': 60, 'P65': 61, 'P66': 62, 'P67': 63, 'P68': 64, 'P69': 65, 'P7': 66, 'P70': 67, 'P71': 68, 'P72': 69, 'P73': 70, 'P74': 71, 'P75': 72, 'P8': 73, 'P9': 74}
Found 3105 images belonging to 75 classes.
[ 0  0  0 ... 74 74 74]
{'P1': 0, 'P10': 1, 'P11': 2, 'P12': 3, 'P13': 4, 'P14': 5,

In [17]:
model_new=train_fine_tuned_vgg16_model(model_new,load_epoch_weights,train_data,val_data,batch_size,checkpoint_path)

C:\Users\PRATHEEK\AppData\Local\Temp\ipykernel_6680\1316762373.py:35: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_new.fit_generator(train_data,


Epoch 1/150
294/294 [==============================] - 3034s 10s/step - loss: 4.0072 - accuracy: 0.1769 - val_loss: 3.4190 - val_accuracy: 0.3351
Epoch 2/150
294/294 [==============================] - 2389s 8s/step - loss: 2.8481 - accuracy: 0.4858 - val_loss: 2.2940 - val_accuracy: 0.6082
Epoch 3/150
294/294 [==============================] - 2482s 8s/step - loss: 1.9625 - accuracy: 0.6549 - val_loss: 1.6644 - val_accuracy: 0.7043
Epoch 4/150


KeyboardInterrupt: 